In [1]:
# 加载readxl包
library(readxl)

# 读取Excel文件
data <- read_excel("5-7.xlsx")

head(data)

y,x1,x2,x3,x4,x5
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
271.8,783.35,33.53,40.55,16.66,13.20
264.0,748.45,36.50,36.19,16.46,14.11
238.8,684.45,34.66,37.31,17.66,15.68
230.7,827.80,33.13,32.52,17.50,10.53
251.6,860.45,35.75,33.71,16.40,11.00
257.9,875.15,34.46,34.14,16.28,11.31


In [4]:
# 定义响应变量Y和自变量X
Y <- data$y
X <- data[, 2:6]

# 设置显著性水平 alpha
alpha <- 0.1

# 初始化空模型（只有截距项）
current_model <- lm(Y ~ 1, data = data)

# 存储已经选择的变量
selected_variables <- c()

# 存储每一步的结果
step_results <- data.frame(Variable = character(), F_value = numeric(), stringsAsFactors = FALSE)

# 获取样本数量
n <- length(Y)

# 循环逐步加入显著性最强的变量
remaining_variables <- colnames(X)  # 初始剩余变量

# 获取当前模型的残差平方和 RSS
get_rss <- function(model) {
  return(sum(residuals(model)^2))
}

# 开始逐步选择
while (length(remaining_variables) > 0) {
  f_values <- sapply(remaining_variables, function(var) {
    # 当前模型的RSS
    rss_current <- get_rss(current_model)
    
    # 为每个剩余变量拟合模型并计算新的RSS
    formula <- as.formula(paste("Y ~", paste(c(selected_variables, var), collapse = " + ")))
    model <- lm(formula, data = data)
    rss_new <- get_rss(model)
    
    # 计算F值
    q <- length(selected_variables)
    f_value <- ((n -  q-2) * (rss_current - rss_new)) / rss_new
    return(f_value)
  })
  
  # 选择F值最大的变量（即引入当前模型后能显著提升模型的变量）
  max_f_value <- max(f_values)
  best_variable <- names(f_values)[which.max(f_values)]
  
  # 计算临界F值（根据F分布，使用显著性水平alpha）
  critical_f_value <- qf(1 - alpha, df1 = 1, df2 = n - length(selected_variables)- 2)
    # 记录当前步骤的结果
    step_results <- rbind(step_results, data.frame(Variable = best_variable, F_value = max_f_value, F_value_need = critical_f_value))
  # 如果F值大于临界F值，则接受该变量
  if (max_f_value > critical_f_value) {
    selected_variables <- c(selected_variables, best_variable)
    remaining_variables <- setdiff(remaining_variables, best_variable)  # 从剩余变量中删除已选的变量
    current_model <- lm(as.formula(paste("Y ~", paste(selected_variables, collapse = " + "))), data = data)
    

  } else {
    break  # 如果没有变量能显著提高F值，停止选择
  }
}

# 打印每一步的变量和F值
print(step_results)

# 打印最终的回归模型
summary(current_model)

  Variable   F_value F_value_need
1       x4 69.609437     2.901192
2       x3 25.430743     2.909132
3       x2  3.060890     2.917745
4       x1  3.688067     2.927117
5       x5  1.876707     2.937356



Call:
lm(formula = as.formula(paste("Y ~", paste(selected_variables, 
    collapse = " + "))), data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-14.322  -2.639   0.025   4.786  16.003 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 270.21013   88.21060   3.063  0.00534 ** 
x4          -21.11940    2.36936  -8.914 4.42e-09 ***
x3            5.33861    0.91506   5.834 5.13e-06 ***
x2            2.95141    1.23167   2.396  0.02471 *  
x1            0.05156    0.02685   1.920  0.06676 .  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 8.17 on 24 degrees of freedom
Multiple R-squared:  0.8909,	Adjusted R-squared:  0.8727 
F-statistic: 48.99 on 4 and 24 DF,  p-value: 3.327e-11


In [27]:
head(data)

y,x1,x2,x3,x4,x5
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
271.8,783.35,33.53,40.55,16.66,13.20
264.0,748.45,36.50,36.19,16.46,14.11
238.8,684.45,34.66,37.31,17.66,15.68
230.7,827.80,33.13,32.52,17.50,10.53
251.6,860.45,35.75,33.71,16.40,11.00
257.9,875.15,34.46,34.14,16.28,11.31


In [6]:
# 定义响应变量Y和自变量X
Y <- data$y
X <- data[, 2:6]

# 设置显著性水平 alpha
alpha <- 0.1

# 初始化空模型（只有截距项）
current_model <- lm(Y ~ 1, data = data)

# 存储已经选择的变量
selected_variables <- c()

# 存储每一步的结果
step_results <- data.frame(Variable = character(), Action = character(), F_value = numeric(), stringsAsFactors = FALSE)

# 获取样本数量
n <- length(Y)

# 获取当前模型的残差平方和 RSS
get_rss <- function(model) {
  return(sum(residuals(model)^2))
}

# 向后剔除和向前选择的逐步回归法
remaining_variables <- colnames(X)  # 初始剩余变量
full_model <- lm(Y ~ ., data = data)  # 全部变量模型

# 开始逐步选择
while (length(remaining_variables)>0) {
  
  # 向前选择：选择最显著的变量加入模型
  f_values_forward <- sapply(remaining_variables, function(var) {
    rss_current <- get_rss(current_model)
    formula <- as.formula(paste("Y ~", paste(c(selected_variables, var), collapse = " + ")))
    model <- lm(formula, data = data)
    rss_new <- get_rss(model)
    q <- length(selected_variables)
    f_value <- ((n - q-2) * (rss_current - rss_new)) /rss_new
    return(f_value)
  })
  
  # 选择最大F值的变量进行加入
  max_f_value_forward <- max(f_values_forward)
  best_variable_forward <- names(f_values_forward)[which.max(f_values_forward)] 
  

  # 计算临界F值（根据F分布，使用显著性水平alpha）
  critical_f_value<- qf(1 - alpha, df1 = 1, df2 = n - length(selected_variables)-2)
    print(critical_f_value)
    print(max_f_value_forward)

  # 如果向前选择的F值大于临界F值，则加入该变量
  if (max_f_value_forward >= critical_f_value) {
    selected_variables <- c(selected_variables, best_variable_forward)
    remaining_variables <- setdiff(remaining_variables, best_variable_forward)  # 从剩余变量中删除已选的变量
    current_model <- lm(as.formula(paste("Y ~", paste(selected_variables, collapse = " + "))), data = data)
    step_results <- rbind(step_results, data.frame(Variable = best_variable_forward, Action = "Add", F_value = max_f_value_forward))
  print(paste("Best variable to add:", best_variable_forward))} else {print('no add')
    # 如果没有变量能显著提高F值，停止选择
    break
  }

    if (length(selected_variables) == 1) {
      next  # 跳过当前循环并进入下一次循环
    }
    
  # 向后剔除：检查每个已选变量，考虑剔除最不显著的变量
  f_values_backward <- sapply(selected_variables, function(var) {
    rss_current <- get_rss(current_model)
    remaining_selected_vars <- setdiff(selected_variables, var)
    formula <- as.formula(paste("Y ~", paste(remaining_selected_vars, collapse = " + ")))
    model <- lm(formula, data = data)
    rss_new <- get_rss(model)
    q <- length(selected_variables)
    f_value <- ((n - q-1) * (rss_new- rss_current )) / rss_current
    return(f_value)
  })
  
  # 计算临界F值（根据F分布，使用显著性水平alpha）
  critical_f_value_backward <- qf(1 - alpha, df1 = 1, df2 = n - length(selected_variables)-1)

  # 选择最大F值的变量进行剔除
  min_f_value_backward <- min(f_values_backward)
  worst_variable_backward <- names(f_values_backward)[which.min(f_values_backward)]
    print(critical_f_value_backward)
    print(min_f_value_backward)
  # 如果向后剔除的F值小于临界F值，则剔除该变量
  if (min_f_value_backward < critical_f_value_backward) {
    selected_variables <- setdiff(selected_variables, worst_variable_backward)
    current_model <- lm(as.formula(paste("Y ~", paste(selected_variables, collapse = " + "))), data = data)
    step_results <- rbind(step_results, data.frame(Variable = worst_variable_backward, Action = "Remove", F_value = max_f_value_backward))
  print(paste("Worst variable to remove:", worst_variable_backward))} else{print('no remove')}
}

# 打印每一步的变量和F值
print(step_results)

# 打印最终的回归模型
summary(current_model)

[1] 2.901192
[1] 69.60944
[1] "Best variable to add: x4"
[1] 2.909132
[1] 25.43074
[1] "Best variable to add: x3"
[1] 2.909132
[1] 25.43074
[1] "no remove"
[1] 2.917745
[1] 3.06089
[1] "Best variable to add: x2"
[1] 2.917745
[1] 3.06089
[1] "no remove"
[1] 2.927117
[1] 3.688067
[1] "Best variable to add: x1"
[1] 2.927117
[1] 3.688067
[1] "no remove"
[1] 2.937356
[1] 1.876707
[1] "no add"
  Variable Action   F_value
1       x4    Add 69.609437
2       x3    Add 25.430743
3       x2    Add  3.060890
4       x1    Add  3.688067



Call:
lm(formula = as.formula(paste("Y ~", paste(selected_variables, 
    collapse = " + "))), data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-14.322  -2.639   0.025   4.786  16.003 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 270.21013   88.21060   3.063  0.00534 ** 
x4          -21.11940    2.36936  -8.914 4.42e-09 ***
x3            5.33861    0.91506   5.834 5.13e-06 ***
x2            2.95141    1.23167   2.396  0.02471 *  
x1            0.05156    0.02685   1.920  0.06676 .  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 8.17 on 24 degrees of freedom
Multiple R-squared:  0.8909,	Adjusted R-squared:  0.8727 
F-statistic: 48.99 on 4 and 24 DF,  p-value: 3.327e-11


In [7]:
# 假设 data 是你的数据框，Y 是响应变量，X1, X2, X3, X4 是自变量
Y <- data$y
X <- data[, c("x1", "x2", "x3", "x4","x5")]

# 获取所有可能的变量组合
all_combinations <- unlist(lapply(1:length(X), function(i) combn(names(X), i, simplify = FALSE)), recursive = FALSE)

# 定义一个空的结果表格，用于存储模型的结果
results <- data.frame(
  Variables = character(),
  RMSq = numeric(),
  AIC = numeric(),
  BIC = numeric(),
  stringsAsFactors = FALSE
)

# 定义计算RMSq的函数
calculate_rmsq <- function(model) {
  rss <- sum(residuals(model)^2)  # 计算残差平方和
  n <- length(model$fitted.values)  # 样本数量
  q <- length(coef(model))  # 模型的参数个数（包括截距项）
  return(rss / (n - q))  # 返回RMSq
}

# 遍历所有组合，建立回归模型，并计算 RMSq、AIC和 BIC
for (comb in all_combinations) {
  formula <- as.formula(paste("Y ~", paste(comb, collapse = " + ")))
  
  # 拟合回归模型
  model <- lm(formula, data = data)
  
  # 计算 RMSq
  rmsq_value <- calculate_rmsq(model)
  
  # 计算 AIC
  aic_value <- AIC(model)
  
  
  # 计算 BIC (贝叶斯信息准则)
  bic_value <- BIC(model)
  
  # 将结果存入表格
  results <- rbind(results, data.frame(
    Variables = paste(comb, collapse = ", "),
    RMSq = rmsq_value,
    AIC = aic_value,
    BIC = bic_value
  ))
}

# 打印结果表格
results

Variables,RMSq,AIC,BIC
<chr>,<dbl>,<dbl>,<dbl>
x1,329.56104,254.3612,258.4631
x2,538.07856,268.5783,272.6801
x3,536.94287,268.5170,272.6189
x4,151.97199,231.9133,236.0152
x5,477.68754,265.1259,269.2278
"x1, x2",311.01665,253.5872,259.0564
"x1, x3",323.92609,254.7666,260.2358
"x1, x4",150.28610,232.4953,237.9645
"x1, x5",341.99399,256.3407,261.8098


In [77]:
model <- lm(Y ~ x3 + x4, data = data)
summary(model)


Call:
lm(formula = Y ~ x3 + x4, data = data)

Residuals:
     Min       1Q   Median       3Q      Max 
-15.2773  -3.5636  -0.9829   4.8435  23.5852 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 483.6703    39.5671  12.224 2.78e-12 ***
x3            4.7963     0.9511   5.043 3.00e-05 ***
x4          -24.2150     1.9405 -12.479 1.75e-12 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 8.932 on 26 degrees of freedom
Multiple R-squared:  0.8587,	Adjusted R-squared:  0.8478 
F-statistic: 79.01 on 2 and 26 DF,  p-value: 8.938e-12


In [78]:
model <- lm(Y ~ x1+x3 + x4, data = data)
summary(model)


Call:
lm(formula = Y ~ x1 + x3 + x4, data = data)

Residuals:
     Min       1Q   Median       3Q      Max 
-14.8389  -5.4673   0.3129   4.4274  23.5650 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 437.32342   58.91498   7.423 8.95e-08 ***
x1            0.02908    0.02744   1.060    0.299    
x3            4.69308    0.95384   4.920 4.58e-05 ***
x4          -22.53409    2.50281  -9.004 2.55e-09 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 8.911 on 25 degrees of freedom
Multiple R-squared:  0.8648,	Adjusted R-squared:  0.8486 
F-statistic:  53.3 on 3 and 25 DF,  p-value: 5.275e-11


In [9]:
model <- lm(Y ~x1+x2+x3 + x4, data = data)
summary(model)


Call:
lm(formula = Y ~ x1 + x2 + x3 + x4, data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-14.322  -2.639   0.025   4.786  16.003 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 270.21013   88.21060   3.063  0.00534 ** 
x1            0.05156    0.02685   1.920  0.06676 .  
x2            2.95141    1.23167   2.396  0.02471 *  
x3            5.33861    0.91506   5.834 5.13e-06 ***
x4          -21.11940    2.36936  -8.914 4.42e-09 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 8.17 on 24 degrees of freedom
Multiple R-squared:  0.8909,	Adjusted R-squared:  0.8727 
F-statistic: 48.99 on 4 and 24 DF,  p-value: 3.327e-11


In [8]:
model <- lm(Y ~ x1+x2+x3 + x4+x5, data = data)
summary(model)


Call:
lm(formula = Y ~ x1 + x2 + x3 + x4 + x5, data = data)

Residuals:
     Min       1Q   Median       3Q      Max 
-13.7049  -2.8200   0.6207   3.9062  17.3937 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 326.37699   95.85338   3.405  0.00243 ** 
x1            0.06780    0.02891   2.345  0.02803 *  
x2            2.54722    1.24523   2.046  0.05240 .  
x3            3.77078    1.45520   2.591  0.01633 *  
x4          -22.97522    2.69281  -8.532  1.4e-08 ***
x5            2.45572    1.79259   1.370  0.18394    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 8.025 on 23 degrees of freedom
Multiple R-squared:  0.8991,	Adjusted R-squared:  0.8772 
F-statistic:    41 on 5 and 23 DF,  p-value: 1.034e-10
